In [2]:
# Dependencies and Setup
#import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
#import requests
#import gmaps
import os
#import json

In [3]:
#location = "Atlanta_crimedata/COBRA-YTD2017.csv"
location = "COBRA-2009-2019.csv"
crimes_df = pd.read_csv(location)
crimes_df.info()

C:\Users\NIC\Anaconda3\envs\pythonData\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342914 entries, 0 to 342913
Data columns (total 19 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Report Number            342914 non-null  int64  
 1   Report Date              342914 non-null  object 
 2   Occur Date               342914 non-null  object 
 3   Occur Time               342914 non-null  object 
 4   Possible Date            342896 non-null  object 
 5   Possible Time            342895 non-null  float64
 6   Beat                     342890 non-null  float64
 7   Apartment Office Prefix  10094 non-null   object 
 8   Apartment Number         68274 non-null   object 
 9   Location                 342912 non-null  object 
 10  Shift Occurence          342914 non-null  object 
 11  Location Type            333698 non-null  object 
 12  UCR Literal              342914 non-null  object 
 13  UCR #                    342914 non-null  int64  
 14  IBR 

In [4]:
crimes_df.head()

,Report Number,Report Date,Occur Date,Occur Time,Possible Date,Possible Time,Beat,Apartment Office Prefix,Apartment Number,Location,Shift Occurence,Location Type,UCR Literal,UCR #,IBR Code,Neighborhood,NPU,Latitude,Longitude
0,90010930,2009-01-01,2009-01-01,1145,2009-01-01,1148.0,411.0,NaN,NaN,2841 GREENBRIAR PKWY,Day Watch,8,LARCENY-NON VEHICLE,630,2303,Greenbriar,R,33.68845,-84.49328
1,90011083,2009-01-01,2009-01-01,1330,2009-01-01,1330.0,511.0,NaN,NaN,12 BROAD ST SW,Day Watch,9,LARCENY-NON VEHICLE,630,2303,Downtown,M,33.75320,-84.39201
2,90011208,2009-01-01,2009-01-01,1500,2009-01-01,1520.0,407.0,NaN,NaN,3500 MARTIN L KING JR DR SW,Unknown,8,LARCENY-NON VEHICLE,630,2303,Adamsville,H,33.75735,-84.50282
3,90011218,2009-01-01,2009-01-01,1450,2009-01-01,1510.0,210.0,NaN,NaN,3393 PEACHTREE RD NE,Evening Watch,8,LARCENY-NON VEHICLE,630,2303,Lenox,B,33.84676,-84.36212
4,90011289,2009-01-01,2009-01-01,1600,2009-01-01,1700.0,411.0,NaN,NaN,2841 GREENBRIAR PKWY SW,Unknown,8,LARCENY-NON VEHICLE,630,2303,Greenbriar,R,33.68677,-84.49773


In [5]:
len(crimes_df["Report Number"].unique())

342609

In [6]:
#removing unnecessary columns
crimes1_df = crimes_df.drop(["Report Date","Possible Date","Possible Time",
                                   "Beat","Apartment Office Prefix","Apartment Number",
                                   "Shift Occurence","Location Type","IBR Code","NPU"],axis=1)

crimes1_df

,Report Number,Occur Date,Occur Time,Location,UCR Literal,UCR #,Neighborhood,Latitude,Longitude
0,90010930,2009-01-01,1145,2841 GREENBRIAR PKWY,LARCENY-NON VEHICLE,630,Greenbriar,33.68845,-84.49328
1,90011083,2009-01-01,1330,12 BROAD ST SW,LARCENY-NON VEHICLE,630,Downtown,33.75320,-84.39201
2,90011208,2009-01-01,1500,3500 MARTIN L KING JR DR SW,LARCENY-NON VEHICLE,630,Adamsville,33.75735,-84.50282
3,90011218,2009-01-01,1450,3393 PEACHTREE RD NE,LARCENY-NON VEHICLE,630,Lenox,33.84676,-84.36212
4,90011289,2009-01-01,1600,2841 GREENBRIAR PKWY SW,LARCENY-NON VEHICLE,630,Greenbriar,33.68677,-84.49773
...,...,...,...,...,...,...,...,...,...
342909,193652089,2019-12-31,2030,1385 SHARON ST NW,AGG ASSAULT,420,Hunter Hills,33.75486,-84.43287
342910,193650336,2019-12-31,432,262 PHARR RD NE,AGG ASSAULT,410,Buckhead Village,33.83732,-84.37860
342911,193650603,2019-12-31,920,689 CASCADE AVE SW,AGG ASSAULT,410,Westview,33.73636,-84.43680
342912,193651760,2019-12-31,1853,763 CASCADE AVE SW,AGG ASSAULT,410,Westview,33.73483,-84.43750


In [7]:
crimes1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342914 entries, 0 to 342913
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Report Number  342914 non-null  int64  
 1   Occur Date     342914 non-null  object 
 2   Occur Time     342914 non-null  object 
 3   Location       342912 non-null  object 
 4   UCR Literal    342914 non-null  object 
 5   UCR #          342914 non-null  int64  
 6   Neighborhood   330551 non-null  object 
 7   Latitude       342914 non-null  float64
 8   Longitude      342914 non-null  float64
dtypes: float64(2), int64(2), object(5)
memory usage: 23.5+ MB


In [8]:
#deleting the rows with null values
cleaned_crimes_df=crimes1_df.dropna(axis=0,how='any')
cleaned_crimes_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 330549 entries, 0 to 342913
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Report Number  330549 non-null  int64  
 1   Occur Date     330549 non-null  object 
 2   Occur Time     330549 non-null  object 
 3   Location       330549 non-null  object 
 4   UCR Literal    330549 non-null  object 
 5   UCR #          330549 non-null  int64  
 6   Neighborhood   330549 non-null  object 
 7   Latitude       330549 non-null  float64
 8   Longitude      330549 non-null  float64
dtypes: float64(2), int64(2), object(5)
memory usage: 25.2+ MB


In [9]:
#Dictionary of Median income per neighborhood
income_dict = {"Paces": 166.6,
              "Brookwood Hills": 148.6,
              "North Buckhead": 128.3,
              "Morningside/Lenox Park": 127.4,
              "Lake Claire": 111.9,
              "Virginia Highland": 93.7,
              "Oakhurst": 92.8,
               "Candler Park": 92.4,
               "Grant Park": 90.9,
               "Ormewood Park": 87.2,
               "Inman Park": 84.3,
               "Peachtree Hills": 83.1,
               "Pine Hills": 80.9,
               "Garden Hills": 79.1,
               "Midtown": 77.6,
               "Bolton": 76.4,
               "Peachtree Heights West": 74.1,
               "Knight Park/Howell Station": 69.7,
               "Kirkwood": 69.1,
               "Underwood Hills": 67.9,
               "East Atlanta": 64.8,
               "Atlanta Area": 59.2,
               "Fulton": 58.9,
               "Linridge/Martin Manor": 58.7,
               "Lindbergh": 57.5,
               "United States": 55.3,
               "South Atlantic": 52.9,
               "Home Park": 52.0,
               "South": 51.2,
               "Georgia": 51.0,
               "Atlanta": 49.4,
               "Downtown": 45.1,
               "Chosewood Park": 41.7,
               "Old Fourth Ward": 38.9,
               "Edgewood": 38.6,
               "West View": 34.9,
               "Venetian Hills": 31.3,
               "Southwest": 30.0,
               "Glenrose Heights": 29.6,
               "Georgia Tech": 28.8,
               "Grove Park": 28.2,
               "Center Hill": 27.7,
               "West End": 27.6,
               "Atlanta University Center": 27.3,
               "Collier Heights": 26.6,
               "Peoplestown": 26.4,
               "Vine City": 25.6,
               "Sylvan Hills": 25.4,
               "Pittsburgh": 22.3,
               "Greenbriar": 21.8,
               "Fairburn Mays": 21.7,
               "English Ave": 21.4,
               "Oakland City": 20.4,
               "Hammond Park": 20.1,
               "Mechanicsvills": 18.3,
               "Campbellton Road": 14.6,
               "Thomasville Heights": 12.2
              }

In [3]:
#crime dictionary
crime_dict = {"HOMICIDE": 110,
            "MANSLAUGHTER": 120,
            "ROBBERY-PEDESTRIAN": 311,
            "ROBBERY-COMMERCIAL": 312,
            "ROBBERY-COMMERCIAL": 313,
            "ROBBERY-COMMERCIAL": 314,
            "ROBBERY-RESIDENCE": 315,
            "ROBBERY-COMMERCIAL": 316,
            "ROBBERY-PEDESTRIAN": 317,
            "ROBBERY-PEDESTRIAN": 321,
            "ROBBERY-COMMERCIAL": 322,
            "ROBBERY-COMMERCIAL": 323,
            "ROBBERY-COMMERCIAL": 324,
            "ROBBERY-RESIDENCE": 325,
            "ROBBERY-COMMERCIAL": 326,
            "ROBBERY-PEDESTRIAN": 327,
            "ROBBERY-PEDESTRIAN": 331,
            "ROBBERY-COMMERCIAL": 332,
            "ROBBERY-COMMERCIAL": 333,
            "ROBBERY-COMMERCIAL": 334,
            "ROBBERY-RESIDENCE": 335,
            "ROBBERY-COMMERCIAL": 336,
            "ROBBERY-PEDESTRIAN": 337,
            "ROBBERY-PEDESTRIAN": 341,
            "ROBBERY-COMMERCIAL": 342,
            "ROBBERY-COMMERCIAL": 343,
            "ROBBERY-COMMERCIAL": 344,
            "ROBBERY-RESIDENCE": 345,
            "ROBBERY-COMMERCIAL": 346,
            "ROBBERY-PEDESTRIAN": 347,
            "AGG ASSAULT": 410,
            "AGG ASSAULT": 420,
            "AGG ASSAULT": 430,
            "AGG ASSAULT": 440,
            "BURGLARY-RESIDENCE": 511,
            "BURGLARY-NONRES": 512,
            "BURGLARY-RESIDENCE": 521,
            "BURGLARY-NONRES": 522,
            "BURGLARY-RESIDENCE": 531,
            "BURGLARY-NONRES": 532,
            "LARCENY-NON VEHICLE": 610,
            "LARCENY-NON VEHICLE": 620,
            "LARCENY-NON VEHICLE": 630,
            "LARCENY-FROM VEHICLE": 640,
            "LARCENY-FROM VEHICLE": 650,
            "LARCENY-NON VEHICLE": 660,
            "LARCENY-NON VEHICLE": 670,
            "LARCENY-NON VEHICLE": 680,
            "LARCENY-NON VEHICLE": 690,
            "AUTO THEFT": 710,
            "AUTO THEFT": 720,
            "AUTO THEFT": 730
}

In [33]:
#Median income dataframe
income_df = pd.DataFrame(list(income_dict.items()), columns = ["Neighborhood", "Median Income (K)"])
income_df.head()


,Neighborhood,Median Income (K)
0,Paces,166.6
1,Brookwood Hills,148.6
2,North Buckhead,128.3
3,Morningside/Lenox Park,127.4
4,Lake Claire,111.9


In [36]:
income_nb = income_df["Neighborhood"]
crime_nb = cleaned_crimes_df["Neighborhood"].unique()
crime_nb



array(['Greenbriar', 'Downtown', 'Adamsville', 'Lenox', 'Edgewood',
       'Venetian Hills', 'Kirkwood', 'Buckhead Forest', 'Candler Park',
       'Old Fourth Ward', 'Midtown', 'Wilson Mill Meadows',
       'Scotts Crossing', 'Chastain Park', 'Collier Heights',
       'Amal Heights', 'Inman Park', 'Capitol View', 'West End',
       'Oakland City', 'Baker Hills', 'Berkeley Park', 'Tuxedo Park',
       'Springlake', 'Fairburn Heights', 'Summerhill', 'South Atlanta',
       'Green Forest Acres', 'The Villages at Carver', 'East Atlanta',
       'Browns Mill Park', 'Sylvan Hills', 'Home Park', 'Dixie Hills',
       'Lake Claire', 'Peoplestown', 'Westview', 'Blandtown',
       'Cascade Avenue/Road', 'Penelope Neighbors', 'Pittsburgh',
       'Pomona Park', 'Buckhead Village', 'Florida Heights', 'Grove Park',
       'Mechanicsville', 'Ashview Heights', 'Atlanta Industrial Park',
       'Lakewood', 'Thomasville Heights', 'Glenrose Heights',
       'Ormewood Park', 'Pine Hills', 'Campbellton Ro

In [52]:
merged_df = pd.merge(cleaned_crimes_df, income_df, how = "outer", on="Neighborhood")
final_df = merged_df.dropna(axis = 0, how = "any")
final_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 177064 entries, 0 to 329773
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Report Number      177064 non-null  float64
 1   Occur Date         177064 non-null  object 
 2   Occur Time         177064 non-null  object 
 3   Location           177064 non-null  object 
 4   UCR Literal        177064 non-null  object 
 5   UCR #              177064 non-null  float64
 6   Neighborhood       177064 non-null  object 
 7   Latitude           177064 non-null  float64
 8   Longitude          177064 non-null  float64
 9   Median Income (K)  177064 non-null  float64
dtypes: float64(5), object(5)
memory usage: 14.9+ MB
